# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sandnessjoen,66.0217,12.6316,4.33,60,100,9.27,NO,1732409790
1,1,tazovsky,67.4667,78.7000,-12.10,94,51,3.94,RU,1732409790
2,2,upper grand lagoon,30.1633,-85.7408,13.34,83,0,0.00,US,1732409790
3,3,alice springs,-23.7000,133.8833,22.75,73,100,5.66,AU,1732409790
4,4,olonkinbyen,70.9221,-8.7187,-3.86,55,21,8.04,SJ,1732409790


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
#coordinates_df = pd.DataFrame(city_data_df)

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 600,
    frame_height = 400,
    alpha = 0.5,
    size = "Humidity",
    scale = 0.7,
    color = 'City'
)

# Display the map
#coordinates_df.head()

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
(city_data_df["Max Temp"] >= 21) &
(city_data_df["Max Temp"] <= 35) &
(city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,atafu village,-8.5421,-172.5159,28.85,70,26,8.60,TK,1732409790
14,14,ribas do rio pardo,-20.4431,-53.7592,24.85,62,0,1.78,BR,1732409791
17,17,port mathurin,-19.6833,63.4167,24.47,77,7,8.51,MU,1732409792
31,31,phai sali,15.5976,100.6585,24.23,74,44,2.46,TH,1732409794
33,33,wailua homesteads,22.0669,-159.3780,25.44,75,0,3.60,US,1732409795


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy([["City", "Country", "Lat", "Lng", "Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,atafu village,-8.5421,-172.5159,28.85,70,26,8.60,TK,1732409790,
14,14,ribas do rio pardo,-20.4431,-53.7592,24.85,62,0,1.78,BR,1732409791,
17,17,port mathurin,-19.6833,63.4167,24.47,77,7,8.51,MU,1732409792,
31,31,phai sali,15.5976,100.6585,24.23,74,44,2.46,TH,1732409794,
33,33,wailua homesteads,22.0669,-159.3780,25.44,75,0,3.60,US,1732409795,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atafu village - nearest hotel: No hotel found
ribas do rio pardo - nearest hotel: Hotel São Francisco
port mathurin - nearest hotel: Escale Vacances
phai sali - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lorengau - nearest hotel: Seeadler Bay Hotel
loango - nearest hotel: Hotel des Aigles
kapa'a - nearest hotel: Pono Kai Resort
remire-montjoly - nearest hotel: Complexe Belova
avarua - nearest hotel: Paradise Inn
tolanaro - nearest hotel: Hôtel Mahavokey
dhuusamarreeb - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
fortuna foothills - nearest hotel: Comfort Inn
nguigmi - nearest hotel: Guest PAM
holualoa - nearest hotel: Kona Hotel
acarau - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
kirakira - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
coahuayana de hidalgo - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
san luis - 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,atafu village,-8.5421,-172.5159,28.85,70,26,8.60,TK,1732409790,No hotel found
14,14,ribas do rio pardo,-20.4431,-53.7592,24.85,62,0,1.78,BR,1732409791,Hotel São Francisco
17,17,port mathurin,-19.6833,63.4167,24.47,77,7,8.51,MU,1732409792,Escale Vacances
31,31,phai sali,15.5976,100.6585,24.23,74,44,2.46,TH,1732409794,No hotel found
33,33,wailua homesteads,22.0669,-159.3780,25.44,75,0,3.60,US,1732409795,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...,...,...,...,...,...
549,549,phetchaburi,12.9667,99.5833,21.71,80,38,1.97,TH,1732409878,resort sunee
550,550,sal rei,16.1833,-22.9167,25.22,70,3,4.76,CV,1732409878,Hotel Marine Club
555,555,imperatriz,-5.5264,-47.4917,27.37,55,2,1.66,BR,1732409879,La Bella Hotel
556,556,medina,24.4686,39.6142,26.24,50,0,1.54,SA,1732409879,The Oberoi Madina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width = 600,
    frame_height = 400,
    alpha = 0.5,
    size = "Humidity",
    scale = 0.7,
    color = 'City',
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)